In [ ]:


from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.models as models
from PIL import Image
import numpy as np
import os

from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import cv2




path_model = 'D:/Christof/trained_networks/210618_adjusted_on_new_img_211008.pth' 
##read roi file with the regions marked as muscular layer to array and create list of all 
arr_results = []
path_roi = 'D:/Christof/full_slides_ext/Annotation_Slides/'

list_roi = os.listdir(path_roi)

for fname in list_roi:
    ##selction of the stain
    if 'ho1' in fname: 
        path_temp = path_roi + fname
        ##load the coordinations of the muscular layer in the whole slide scan from geojson
        with open(path_temp) as f:
            lines = f.readlines()
        if len(lines) > 3:
            arr_coord = np.array([])    

            coord_list = lines[8:-9]

            arr_coord = np.zeros((len(coord_list), 2))

            for coord in coord_list:
                index = coord_list.index(coord)
                coord = coord[11:-2]
                if coord[-1] == ']':
                    coord = coord[:-1]
                for char in coord:
                    if char == ',':
                        ind = coord.index(char)
                        y = int(coord[:ind])
                        x = int(coord[ind+1:])

                arr_coord[index] = y,x
            arr_coord = arr_coord.astype(int)


            fname_img = fname[:-4]

            ##load the whole slide images as tif
            from skimage import io
            import matplotlib.pyplot as plt
            import numpy as np
            import cv2


            path = 'D:/Christof/full_slides_ext/slides_tif/'
            path = path + fname[:-9] + '/' + fname[:-9]+'_Wholeslide_Default_Extended.tif'

            print(path)
            img = io.imread(path)
            pts = arr_coord

            ##calculate area of the annotation
            
            def PolygonArea(corners):
                n = len(corners) # of corners
                area = 0.0
                for i in range(n):
                    j = (i + 1) % n
                    area += corners[i][0] * corners[j][1]
                    area -= corners[j][0] * corners[i][1]
                area = abs(area) / 2.0
                return area
                print(area)


            area = PolygonArea(pts)

            area_mm2 = area * 0.000274909**2
            
            
            ##create bounding rectangle for creating the mask
            
            rect = cv2.boundingRect(pts)
            x,y,w,h = rect
            im = img[y:y+h, x:x+w].copy()


            ##create mask for classifying the relevant areas
            pts = pts - pts.min(axis=0)

            mask = np.zeros(im.shape[:2], np.uint8)
            mask = cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

            im = Image.fromarray(im, 'RGB')
            shape_im = np.shape(im)

            x_size = shape_im[1]
            y_size = shape_im[0]

            ##definition of the step size and the number of steps needed to analyse the whole muscular layer
            stepsize = 20
            stepsis = stepsize

            steps_x = x_size//stepsis -70//stepsize +1
            rest_x = x_size%stepsize

            steps_y = y_size//stepsis -70//stepsize +1
            rest_y = y_size%stepsize

            print('steps in x',steps_x, rest_x)
            print('steps in y', steps_y, rest_y)

            ##definition of the arrays for the results of the classification and the predicition
            arr_result = np.zeros((steps_y, steps_x))
            arr_sigmoid = np.zeros((steps_y, steps_x))
            arr_pred = np.zeros((steps_y, steps_x))
            
            #create ductuibary with the coordinations inside the muscular layer mask
            dict_img_coord = {}

            
            n=0
            for y in range(0, steps_y):
                
                for x in range(0, steps_x):
                    x_coord = int(x*stepsize)
                    y_coord = int(y*stepsize)

                    if mask[y_coord, x_coord] > 0:
                        dict_img_coord[n] = (int(x * stepsize), int(y *stepsize))
                        #x,y = dict_img_coord.get(n) 
                        n +=1
            len_dict_coord = len(dict_img_coord)
           

            print('number of tiles to classify' ,len_dict_coord)

            ##loading of AI model

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            model = torchvision.models.resnet50()
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 2)
            model.load_state_dict(torch.load(path_model))
            model.eval()
            model.to(device)

            ##definition of the image loader

            loader = transforms.Compose([transforms.Resize((70,70)), transforms.ToTensor()])

            def image_loader(image):
                """load image, returns cuda tensor"""
                image = loader(image).float()
                image = image.unsqueeze_(0).cuda()  #this is for VGG, may not be needed for ResNet
                return image.cuda()  #assumes that you're using GPU 

            def add_margin(pil_img, top, right, bottom, left, color):
                width, height  = pil_img.size
                new_width = width + right + left
                new_height = height + top + bottom
                result = Image.new(pil_img.mode, (new_width, new_height), color)
                result.paste(pil_img, (left, top))
                return result

            ##classify images at all selected coordinations in the muscular layer and save the predictions in an array
            for i in tqdm(range(0, len_dict_coord)):
                coord = dict_img_coord.get(i)
                x = coord[0]
                y = coord[1]
                left = x 
                top = y + 70
                right = x + 70
                bottom = y
                im1 = im.crop((left, bottom, right, top, ))
                fname = str(i) + '.tif'
                image = image_loader(im1)
                pred = model(image)
                pred = val_pos = float(pred[0][1].cpu())
                x_arr = int(x/stepsize)
                y_arr = int(y/stepsize)
                arr_result[y_arr, x_arr] = pred
                arr_sigmoid[y_arr, x_arr] = 1/(1 + np.exp(-pred))



            ##array with binary predictions
            arr_pred = np.zeros((steps_y, steps_x ))
            ##smoothened array with singel positives sorted out
            arr_smooth = np.zeros((steps_y , steps_x ))


            y_sig, x_sig = np.shape(arr_sigmoid)

            ##stetting the threshold for the prediction
            thresh = 0.55
            for y in range(0, y_sig):
                for x in range(0, x_sig):
                    if arr_sigmoid[y,x] > thresh:
                        arr_pred[y, x] = 1

            ##smoothing the array
            y_sig, x_sig = np.shape(arr_sigmoid)

            smooth_area = 1
            thresh_smooth = 0
            for y in range(0, y_sig):
                for x in range(0, x_sig):
                    smooth = np.sum(arr_pred[y-smooth_area:y+smooth_area+1,x-smooth_area:x+smooth_area+1])
                    if smooth > thresh_smooth:
                        arr_smooth[y,x] = smooth




            #check if in the sourrounding pixels are higher values for segmentation 

            list_test = []

            arr_max = np.zeros((steps_y , steps_x ))
            x_sm, y_sm = np.shape(arr_smooth)
            
            for y in range(0, y_sm -1):
                for x in range(0, x_sm-1):
                    list_neighbours = [arr_smooth[x-1,y-1], arr_smooth[x-1,y], arr_smooth[x-1,y+1], arr_smooth[x,y-1], arr_smooth[x,y+1], arr_smooth[x+1,y-1], arr_smooth[x+1,y], arr_smooth[x+1,y+1] ]
                    val = arr_smooth[x,y]
                    if val> 1.0:
                        for neighbor in list_neighbours:
                            if val < neighbor:
                                val = 0
                                arr_max[x,y] = 0.0
                        if val > 0:
                            arr_max[x,y] = 1.0
                            list_test.append((x,y))
                    else:
                        arr_max[x,y] = 0.0

            list_max = []

            #combine the boxes, if they lay directly next to each other

            for coord in list_test:
                x = coord[0]
                y = coord[1]
                list_surround2 = []
                list_surround = [(x+1, y), (x+1, y+1), (x, y+1)]
                for coord2 in list_surround: 
                    x = coord2[0]
                    y = coord2[1]
                    list_auxil = [(x+1, y), (x-1,y),(x+1,y-1), (x, y-1),(x-1,y-1), (x+1, y+1), (x, y+1),(x-1,y+1)]
                    for coord2 in list_auxil:
                        if coord2 not in list_surround2:
                            list_surround2.append(coord2)

                list_temp = []
                for i in list_surround2:
                    if i in list_test:
                        ind = list_test.index(i)
                        del list_test[ind]
                        list_temp.append(i)
                list_x = []
                list_y = []
                for i in list_temp:
                    list_x.append(i[0])
                    list_y.append(i[1])
                print(list_x)
                new_coord = (np.mean(list_x), np.mean(list_y))
                list_test.append(new_coord)

                print(list_test)

    

            #check again if there are positives next to each other
            for coord in list_test:
                x = coord[0]
                y = coord[1]
                list_surround2 = []
                list_surround = [(x,y),(x+1, y), (x-1,y),(x+1,y-1), (x, y-1),(x-1,y-1), (x+1, y+1), (x, y+1),(x-1,y+1)]

                #print(list_surround)
                list_temp = []
                
                for i in list_surround:
                    if i in list_test:
                        ind = list_test.index(i)
                        del list_test[ind]
                        list_temp.append(i)
                        #print(list_temp)
                list_x = []
                list_y = []
                for i in list_temp:
                    list_x.append(i[0])
                    list_y.append(i[1])
                #print(np.mean(list_x))
                new_coord = (np.mean(list_x), np.mean(list_y))
                list_test.append(new_coord)
           
            ##check if there are cell neighbouring directly another using net with lower threshold
            for i in tqdm(range(0, len(list_test))):
                coord = list_test[i]
                y = int(coord[0]*stepsize)
                x = int(coord[1]*stepsize)
                count_pos = 0
                for n in range(0,3):
                    for k in range(0,3):
                        if k != 1:
                            y_temp = y-45+45*k
                            x_temp = x-45 + 45*n
                            left = x_temp 
                            top = y_temp + 70
                            right = x_temp + 70
                            bottom = y_temp
                            im1 = im.crop((left, bottom, right, top, ))
                            fname = str(i) + '.tif'
                            image = image_loader(im1)
                            pred = model(image)
                            pred = val_pos = float(pred[0][1].cpu())
                            pred_sig = 1/(1 + np.exp(-pred))
                            thresh_again = thresh *0.8
                            if pred_sig > thresh_again:
                                count_pos += 1
                                list_test.append((x_temp, y_temp))


            ##second round of classification on positive rois with higher threshold for higher accuracy

            list_result = []

            for i in range(0, len(list_test)):
                coord = list_test[i]
                #print(coord)
                y = int(coord[0]*stepsize)
                x = int(coord[1]*stepsize)
                count_pos = 0
                for n in range(0,3):
                    for k in range(0,3):
                        y_temp = y-7 +7*k
                        x_temp = x-7 + 7*n
                        left = x_temp 
                        top = y_temp + 70
                        right = x_temp + 70
                        bottom = y_temp
                        im1 = im.crop((left, bottom, right, top, ))
                        fname = str(i) + '.tif'
                        image = image_loader(im1)
                        pred = model(image)
                        pred = val_pos = float(pred[0][1].cpu())
                        pred_sig = 1/(1 + np.exp(-pred))
                        thresh_again = thresh *1.3
                        if pred_sig > thresh_again:
                            count_pos += 1
                if count_pos > 4:
                    list_result.append(coord)


            arr_results.append([fname_img, area_mm2, str(len(list_result)) ,list_result])
            np.savetxt("210929_fullslideHO1_scan_adjusted_net.csv", arr_results, delimiter =", ", fmt ='% s') 

D:/Christof/full_slides_ext/slides_tif/3604pylho1/3604pylho1_Wholeslide_Default_Extended.tif
steps in x 1850 10
steps in y 842 7
number of tiles to classify 600943
(32580, 20)


In [1]:
arr_results.append([fname_img, str(len(list_result)) ,list_result])
np.savetxt("210927_fullslideHO1_scan.txt", arr_results, delimiter =", ", fmt ='% s') 

NameError: name 'arr_results' is not defined

In [4]:
np.save('211029_fullslide_HO1_scan_0.25,x1.6.npy', arr_result, allow_pickle=True, fix_imports=True)

In [ ]:
print(arr_reuslts)

In [15]:
result_short = []

for slide in arr_results:
    count = float(slide[2])
    area_mm2 = float(slide[1])
    cell_density = count / area_mm2
    list_temp = slide[:3]
    list_temp.append(cell_density)
    result_short.append(list_temp)
    
print(result_short)
    

[['3604pylho1.mrxs', 18.15446612188232, '463', 25.5033663282407], ['3605ppvho1.mrxs', 24.367121400325683, '561', 23.022826159207376], ['3606pglho1.mrxs', 154.3365829703167, '164', 1.0626126148687758], ['3606ppvho1.mrxs', 25.765203147742035, '693', 26.896741160014177], ['3607pglho1.mrxs', 15.241062665243595, '563', 36.939681462231015], ['3607pylho1.mrxs', 15.293558316039869, '243', 15.889042626864791], ['3701pylho1.mrxs', 39.03497120852008, '670', 17.16409617470809], ['3702pglho1.mrxs', 12.029374559651902, '238', 19.784902267345068], ['3702ppvho1.mrxs', 11.668613150737448, '633', 54.2480920245432], ['3703ppvho1.mrxs', 10.738253483878465, '294', 27.378753951132513], ['3704pylho1.mrxs', 292.3610021775231, '205', 0.7011879097182836], ['3708pglho1.mrxs', 19.813545874359665, '392', 19.78444456563829], ['3802pglho1.mrxs', 10.848650851174247, '338', 31.15594783506331], ['3802ppvho1.mrxs', 1.6277639459518443, '24', 14.744152590236824], ['3803pglho1.mrxs', 14.841531264943509, '182', 12.262885597

In [18]:
print(arr_results)

[['3604pylho1.mrxs', '0', []], ['3605ppvho1.mrxs', '0', []], ['3606pglho1.mrxs', '0', []]]
